<h1 style="background:#e6ffff;color:#ffff80"><center>Please Upvote to Support</center></h1>

The Notebook is based on Acea water prediction competition. The competition is closed already! [Click here to view Acea Water Prediction](https://www.kaggle.com/c/acea-water-prediction)
* For beginners you can practice the code by creating new notebook(PS Don't forget to tag me as reference ;)

The Notebook will majorly contain: **Data Visualization**
* **Follow the below table of contents to scroll easily through the notebook** (click the serail numbers to jump to the section)
* Pictures/Photographs used in this Notebook belong to their respective owners

<a id='tc'></a>
* [1](#s1) **Introduction**
* [2](#s2) **Data Visualization**
  * [2.1](#s21) **Loading Data**
  * [2.2](#s22) **Defining Graph**
* [3](#s3) **Aquifer**
* [4](#s4) **Lake**
* [5](#s5) **River**
* [6](#s6) **Water Spring**

<a id='s1'><h2>Section 1</h2></a>
[back to table of contents](#tc)
****
<h1 style="background:#e6ffff;color:#004d4d"><center>Introduction</center></h1>

This competition uses nine different datasets, completely independent and not linked to each other. Each dataset can represent a different kind of waterbody. As each waterbody is different from the other, the related features as well are different from each other. So, if for instance we consider a water spring we notice that its features are different from the lake’s one. This is correct and reflects the behavior and characteristics of each waterbody. The Acea Group deals with four different type of waterbodies: water spring (for which three datasets are provided), lake (for which a dataset is provided), river (for which a dataset is provided) and aquifers (for which four datasets are provided). **Link to download data description is given** [here](https://www.kaggle.com/c/acea-water-prediction/data?select=datasets_description.ods)

![Water](https://www.theheraldtimes.com/wp-content/uploads/2020/04/27-scaled.jpg)



In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<a id='s2'><h2>Section 2</h2></a>
[back to table of contents](#tc)
****
<h1 style="background:#e6ffff;color:#004d4d"><center>Data Visualization</center></h1>

In [ ]:
#loading packages for plotting 
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['figure.dpi'] = 200

#for data preporocessing, and predicting
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor

# ignoring warnings
import warnings
warnings.simplefilter("ignore")

<a id='s21'><h2>Section 2.1</h2></a>
[back to table of contents](#tc)
****
<h1 style= "color:#004d4d">Loading The Data</h1>

In [ ]:
#lets first list the files in acea-water-prediction directory
os.listdir('../input/acea-water-prediction')

#9 csv files, 9 datasets

In [ ]:
#saving csv 
Aquifer_Doganella = pd.read_csv('../input/acea-water-prediction/Aquifer_Doganella.csv', index_col = 'Date')
Aquifer_Auser = pd.read_csv('../input/acea-water-prediction/Aquifer_Auser.csv', index_col = 'Date')
Water_Spring_Amiata = pd.read_csv('../input/acea-water-prediction/Water_Spring_Amiata.csv', index_col = 'Date')
Lake_Bilancino = pd.read_csv('../input/acea-water-prediction/Lake_Bilancino.csv', index_col = 'Date')
Water_Spring_Madonna_di_Canneto = pd.read_csv('../input/acea-water-prediction/Water_Spring_Madonna_di_Canneto.csv', index_col = 'Date')
Aquifer_Luco = pd.read_csv('../input/acea-water-prediction/Aquifer_Luco.csv', index_col = 'Date')
Aquifer_Petrignano = pd.read_csv('../input/acea-water-prediction/Aquifer_Petrignano.csv', index_col = 'Date')
Water_Spring_Lupa = pd.read_csv('../input/acea-water-prediction/Water_Spring_Lupa.csv', index_col = 'Date')
River_Arno = pd.read_csv('../input/acea-water-prediction/River_Arno.csv', index_col = 'Date')


In [ ]:
datasets = [Aquifer_Doganella,Aquifer_Auser,Water_Spring_Amiata,Lake_Bilancino,Water_Spring_Madonna_di_Canneto,Aquifer_Luco,Aquifer_Petrignano,Water_Spring_Lupa,River_Arno]
datasets_names = ['Aquifer_Doganella', 'Aquifer_Auser', 'Water_Spring_Amiata','Lake_Bilancino', 'Water_Spring_Madonna_di_Canneto', 'Aquifer_Luco','Aquifer_Petrignano', 'Water_Spring_Lupa', 'River_Arno']

In [ ]:
#info for each dataset

#[print('{}: \n{}\n -------------------------------'.format(datasets_names[i], datasets[i].dtypes.value_counts())) for i in range(len(datasets))]

In [ ]:
#[print('{}: \n{}\n -------------------------------'.format(datasets_names[i], datasets[i].head(3))) for i in range(len(datasets))]

<a id='s22'><h2>Section 2.2</h2></a>
[back to table of contents](#tc)
****
<h1 style= "color:#004d4d">Defining Graph Functions</h1>

Here we will create a series of graph to understand the data. After every function - there will be a list of the parameters used and their explaination and role in plotting the graph. Some graphs are commented, you can uncomment and run the graphs

In [ ]:
def bar_plot(x, y, title, palette_len, xlim = None, ylim = None, 
             xticklabels = None, yticklabels = None, 
             top_visible = False, right_visible = False, 
             bottom_visible = True, left_visible = False,
             xlabel = None, ylabel = None, figsize = (6, 4),
             axis_grid = 'x'):
    fig, ax = plt.subplots(figsize = figsize)
    plt.title(title)
    
    #borders
    ax.spines['top'].set_visible(top_visible)
    ax.spines['right'].set_visible(right_visible)
    ax.spines['bottom'].set_visible(bottom_visible)
    ax.spines['left'].set_visible(left_visible)
    for i in ['top', 'right', 'bottom', 'left']:
        ax.spines[i].set_color('blue')
    
    #plotting the graph
    sns.barplot(x = x, y = y, edgecolor = 'black', ax = ax,
                palette = sns.color_palette("mako", len(palette_len)))
    
    #formatting the axes
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)    
    ax.set_xticklabels(xticklabels)
    ax.set_yticklabels(yticklabels)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    ax.grid(axis = axis_grid, linestyle = ':', alpha = 0.5) #try changing these values and note down the differences
    plt.show()

* x- data for x axis
* y- data for y axis
* palette_len - Every bar plotted will have a unique color and the number of colors required is defined as palette length (basically it's equal to y)
* xlim, ylim - limit for x and y
* xticklables - https://www.mathworks.com/help/matlab/ref/xticklabels.html (tick lables for x axis)
* yticklables - https://www.mathworks.com/help/matlab/ref/yticklabels.html (tick lables for y axis)
* top_visible - this parameter is used for visibility of top axis . Respectively for right, bottom and left
* xlabel, ylabel - Lable for respective axis
* figsize(x, y) - size of figure
* axis_grid - grid lines (here the dotted lines vertical lines specifiying x lables)


In [ ]:
def columns_viz(data, color="#1DF1C9"):
    for i in range(len(data.columns)):
        line_plot(data=data, y=data.columns[i],
                  color = color,
                  title = '{} dynamics'.format(data.columns[i]),
                  bottom_visible = False, figsize = (10, 3))

Will be used to study the pattern of each column

In [ ]:
colors=['#000080', '#9999ff', '#3399ff', '#cce6ff', '#1f3d7a',' #0000e6']
def line_plot(data, y, title, color,
              top_visible = False, right_visible = False, 
              bottom_visible = True, left_visible = False,
              ylabel = None, figsize = (10, 4), axis_grid = 'y'):
    
    fig, ax = plt.subplots(figsize = figsize)
    plt.title(title, size = 15, fontweight = 'bold', fontfamily = 'serif')
    
    #borders/spines
    for i in ['top', 'right', 'bottom', 'left']:
        ax.spines[i].set_color('black')
    
    ax.spines['top'].set_visible(top_visible)
    ax.spines['right'].set_visible(right_visible)
    ax.spines['bottom'].set_visible(bottom_visible)
    ax.spines['left'].set_visible(left_visible)
    
    sns.lineplot(x=data.index.values, y = data[y], color = color)
    
    ax.set_xticks([])
    plt.xlabel('')
    plt.ylabel(ylabel, fontfamily = 'serif')
    ax.grid(axis = axis_grid, linestyle = '--', alpha = 0.9)
    plt.show()

In [ ]:
def corr_plot(dataset, x):
    corr_temp = dataset[dataset.columns[dataset.columns.str.contains(x)].tolist()].corr()
    mask = np.triu(np.ones_like(corr_temp, dtype=bool))


    fig, ax = plt.subplots(figsize=(10, 8))

    sns.heatmap(corr_temp, mask=mask, annot=True, cmap="Blues")
    plt.show()

In the given datasets, there are 4 types of water bodies. We will evaluate each of these 4 water bodies separately

<a id='s3'><h2>Section 3</h2></a>
[back to table of contents](#tc)
****
<h1 style= "background:#e6ffff;color:#004d4d"><center>Aquifer</center></h1>

![Aquifer](https://upload.wikimedia.org/wikipedia/commons/thumb/0/04/Aquifer_en.svg/350px-Aquifer_en.svg.png)

***

<h2>Aquifer_Doganella, Aquifer_Auser, Aquifer_Luco, Aquifer_Petrignano</h2>

In [ ]:
Aquifer = [Aquifer_Doganella, Aquifer_Auser, Aquifer_Luco, Aquifer_Petrignano]
Aquifer_names = ['Aquifer_Doganella', 'Aquifer_Auser', 'Aquifer_Luco', 'Aquifer_Petrignano']

In [ ]:
for i in range(len(Aquifer)):
    print('{}: \n{}\n Shape- {}\n -------------------------------'.format(Aquifer_names[i], Aquifer[i].dtypes.value_counts(), Aquifer[i].shape))

In [ ]:
for i in range(len(Aquifer)):
    NaN_values = (Aquifer[i].isnull().sum() / len(Aquifer[i]) * 100).sort_values(ascending = False)
    bar_plot(x = NaN_values, 
             y = NaN_values.index,
             title = Aquifer_names[i] + 'NaN values (%)',
             palette_len = NaN_values.index, 
             xlim = (0, 100), 
             xticklabels = range(0, 101, 20),
             yticklabels = NaN_values.index,
             left_visible = True,
             figsize = (10, 8), axis_grid = 'x')

**Uncomment the below line of code and run!**
* It may take some time

In [ ]:
for i in range(len(Aquifer)):
    print(Aquifer_names[i],'*'*30)
    columns_viz(Aquifer[i], color = colors[i])
    print('*'*30)


In [ ]:
for i in range(len(Aquifer)):
    print(Aquifer_names[i],'*'*30)
    corr_plot(Aquifer[i], x="^Depth_to_Groundwater")
    print('*'*30)

<a id='s4'><h2>Section 4</h2></a>
[back to table of contents](#tc)
****
<h1 style= "background:#e6ffff;color:#004d4d"><center>Lake</center></h1>

![Lake](https://www.hoteldeivicari.com/images/demo/gallery/intestazione1280x500/mugello_florence_lake_bilancino_near_florence.jpg)
* Since there is only one lake, we will not make any List or List_names

In [ ]:
Lake_Bilancino.head(3)

In [ ]:
NaN_values = (Lake_Bilancino.isnull().sum() / len(Lake_Bilancino) * 100).sort_values(ascending = False)
bar_plot(x = NaN_values, 
 y = NaN_values.index,
 title = 'Lake_Bilancino NaN values (%)',
 palette_len = NaN_values.index, 
 xlim = (0, 100), 
 xticklabels = range(0, 101, 20),
 yticklabels = NaN_values.index,
 left_visible = True,
 figsize = (10, 8), axis_grid = 'x')

In [ ]:
columns_viz(Lake_Bilancino)

In [ ]:
corr_plot(Lake_Bilancino, x="")

<a id='s5'><h2>Section 5</h2></a>
[back to table of contents](#tc)
****
<h1 style= "background:#e6ffff;color:#004d4d"><center>River</center></h1>

![River](https://www.followyourdetour.com/wp-content/uploads/2020/10/Florence-Arno-River.jpg)

In [ ]:
River_Arno.head(3)

In [ ]:
NaN_values = (River_Arno.isnull().sum() / len(River_Arno) * 100).sort_values(ascending = False)
bar_plot(x = NaN_values, 
 y = NaN_values.index,
 title = 'River_Arno NaN values (%)',
 palette_len = NaN_values.index, 
 xlim = (0, 100), 
 xticklabels = range(0, 101, 20),
 yticklabels = NaN_values.index,
 left_visible = True,
 figsize = (10, 8), axis_grid = 'x')

In [ ]:
columns_viz(River_Arno)

In [ ]:
corr_plot(River_Arno, x="")

<a id='s6'><h2>Section 6</h2></a>
[back to table of contents](#tc)
****
<h1 style= "background:#e6ffff;color:#004d4d"><center>Water Spring</center></h1>

![Water Spring](https://www.toscanaumbriaintreno.com/thumbs/1000x1000/n/territorio/1/5cb9a95b070d8_amiata.jpg)


<h2 style='color:#1f3d7a;'>Water_Spring_Amiata,  Water_Spring_Madonna_di_Canneto, Water_Spring_Lupa</h2>

In [ ]:
Springs = [Water_Spring_Amiata,  Water_Spring_Madonna_di_Canneto, Water_Spring_Lupa]
Springs_names =['Water_Spring_Amiata',  'Water_Spring_Madonna_di_Canneto', 'Water_Spring_Lupa']

In [ ]:
for i in range(len(Springs)):
    NaN_values = (Springs[i].isnull().sum() / len(Springs[i]) * 100).sort_values(ascending = False)
    bar_plot(x = NaN_values, 
             y = NaN_values.index,
             title = Springs_names[i] + 'NaN values (%)',
             palette_len = NaN_values.index, 
             xlim = (0, 100), 
             xticklabels = range(0, 101, 20),
             yticklabels = NaN_values.index,
             left_visible = True,
             figsize = (10, 8), axis_grid = 'x')

In [ ]:
'''for i in range(len(Springs)):
    print(Springs_names[i],'*'*30)
    columns_viz(Springs[i], color = colors[i])
    print('*'*30)
'''



In [ ]:
for i in range(len(Springs)):
    print(Springs_names[i],'*'*30)
    corr_plot(Springs[i], x="")
    print('*'*30)